#### Model 설정
- 가중치(weight), 절편(bias) => 초기값 설정
- 규제 => L1 norm, L2 norm
- 층 => 일부 퍼셉트론을 비활성화 시켜주는 층 ==> Dropout Layer
    * 학습때만 지정된 비율의 퍼셉트론을 비활성화
    * 예측때는 모든 퍼셉트론 활성화
- fit() 학습 시 학습 분량 조절 => 배치 사이즈 (batch_size)
    * 학습 직전에 지정된 학습분량 데이터를 정규화
- 최적화 방법
    * compile()함수의 optimizer 설정

##### [1] 데이터 준비 및 확인
[1-1] 데이터 로딩

In [106]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.datasets.mnist import load_data

import numpy as np
import matplotlib.pyplot as plt

In [107]:
# 과학적표기법 사용X
np.set_printoptions(floatmode='fixed', suppress=True)

In [108]:
(X_tr, y_tr), (X_te, y_te) = load_data()

In [109]:
X_tr.shape, y_tr.shape, X_te.shape, y_te.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

[1-2] 데이터 전처리
- 픽셀값 0 ~ 255 ---> 0.0 ~ 1.0

In [110]:
X_tr = X_tr / X_tr.max()

In [111]:
X_te = X_te / X_te.max()

##### [2] 모델 생성

[2-1] 모델 설계
- 입력층 : Flatten
- 은닉층 : Dense, 120, relu     <-- w, b 초기화? 기본 | 사용자 설정 'he_normal', 'ones'
- 출력층 : Dense , 10, softmax  <-- w, b 초기화? 기본 | 사용자 설정 'lecun_uniform', 'ones'

In [112]:
model = Sequential(name='My_MNIST')

In [113]:
# 입력층 : (28, 28) ===> (784, ) 다음 Layer 전달
model.add(Flatten(input_shape=(28, 28)))

In [114]:
# 은닉층 : (784, ) ===> 50 다음 Layer 전달
model.add(Dense(50,
                kernel_initializer = 'he_normal',
                kernel_regularizer = 'l1',
                bias_initializer='ones',
                activation='relu'))

In [115]:
# 은닉층 : 지정된 비율의 퍼셉트론 비활성화
model.add(Dropout(.3))

In [116]:
# 출력층 : 50 ===> 10 출력
model.add(Dense(10,
                kernel_initializer='lecun_uniform',
                kernel_regularizer = 'l2',
                activation='softmax'))

In [117]:
model.summary()

Model: "My_MNIST"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                39250     
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


[2-2] 모델 생성

In [118]:
model.compile(loss='sparse_categorical_crossentropy', metrics='acc')

##### [3] 학습
- 학습횟수 => epochs
- 학습분량 => batch_size
- 검증여부 => validation_data, validation_split
- 학습 모니터링 => callbacks

[3-1] 콜백 객체들 생성
- 모델체크 : 성능 좋은 모델 저장, 가중치만 저장
- 조기종료 : 성능 개선이 없는 경우 학습 중단
- 학습률체크 : 반복 학습 시 학습률 조정

In [119]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

In [120]:
from datetime import datetime

In [121]:
datetime.now().ctime()

'Wed Mar 22 17:41:10 2023'

In [122]:
FILEPATH = f'./save_model_{datetime.now().ctime()}.ckp'
# FILEPATH = './save_model.ckp'
modelCB = ModelCheckpoint(filepath=FILEPATH, monitor='val_accuracy', save_best_only=True)

In [123]:
stopCB = EarlyStopping(monitor='val_accuracy', patience=2)

In [124]:
model.fit(X_tr, y_tr,
          epochs=10,
          validation_split=0.2,
          callbacks=[modelCB, stopCB])

Epoch 1/10
1500/1500 [==============================] - 2s 845us/step - loss: 2.5201 - acc: 0.6237 - val_loss: 1.8403 - val_acc: 0.8075
Epoch 2/10
1500/1500 [==============================] - 1s 780us/step - loss: 1.9998 - acc: 0.6906 - val_loss: 1.7854 - val_acc: 0.8313
Epoch 3/10
1500/1500 [==============================] - 1s 758us/step - loss: 1.9728 - acc: 0.7021 - val_loss: 1.8108 - val_acc: 0.8269
Epoch 4/10
1500/1500 [==============================] - 1s 868us/step - loss: 1.9588 - acc: 0.7092 - val_loss: 1.7564 - val_acc: 0.8355
Epoch 5/10
1500/1500 [==============================] - 1s 945us/step - loss: 1.9501 - acc: 0.7088 - val_loss: 1.7411 - val_acc: 0.8407
Epoch 6/10
1500/1500 [==============================] - 1s 851us/step - loss: 1.9431 - acc: 0.7083 - val_loss: 1.7543 - val_acc: 0.8379
Epoch 7/10
1500/1500 [==============================] - 1s 902us/step - loss: 1.9350 - acc: 0.7118 - val_loss: 1.7291 - val_acc: 0.8431
Epoch 8/10
1500/1500 [==========================